In [1]:
import os
import cv2
import numpy as np
import easyocr
import re
from concurrent.futures import ThreadPoolExecutor
from openpyxl import Workbook
from collections import Counter
from pathlib import Path

main_folder = "ELEKTROKOTER HATA KODLARI 06.09.2024"
excel_path = "excel_output.xlsx"

# Renk sınır setleri
blue_ranges = [
    (np.array([105, 120, 40]), np.array([145, 255, 255])),
    (np.array([100, 60, 30]), np.array([140, 255, 255])),
    (np.array([90, 20, 20]), np.array([140, 255, 255])),
]


# OCR ile görüntü işleme ve hata kodu tespiti
def process_image(image_path):
    image = cv2.imread(image_path)

    # Renk sınırları deneme
    for lower_blue, upper_blue in blue_ranges:
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
        result = cv2.bitwise_and(image, image, mask=mask)
        gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
        contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        screen_contour = None
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w > 400 and h > 200:
                screen_contour = contour
                break

        if screen_contour is not None:
            x, y, w, h = cv2.boundingRect(screen_contour)
            cropped_image = image[y : y + h, x : x + w]
            gray_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

            # Keskinleştirme filtresi ve kontras
            kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
            sharpened = cv2.filter2D(gray_cropped, -1, kernel)
            alpha = 0.55
            beta = 1.3
            sharpened = cv2.convertScaleAbs(sharpened, alpha=alpha, beta=beta)

            # OCR işlemi
            reader = easyocr.Reader(["en"], gpu=True)
            ocr_result = reader.readtext(sharpened, detail=0)

            # OCR sonuçlarını temizleme
            clean_result = [
                text.upper().replace("€", "C").replace("o", "0").replace("O", "0")
                for text in ocr_result
            ]
            clean_result = [re.sub(r"[^A-Za-z0-9 ]", "", text) for text in clean_result]
            clean_result = ["".join(text.split()) for text in clean_result]

            # Hata kodu tespiti
            matches = []
            i = 0
            while i < len(clean_result):
                if clean_result[i] in ["A", "B", "C", "4"]:
                    if i + 1 < len(clean_result) and re.match(
                        r"^[A-Z0-9]{2}$", clean_result[i + 1]
                    ):
                        combined_code = clean_result[i] + clean_result[i + 1]
                        matches.append(combined_code)
                        i += 1
                    elif len(clean_result[i + 1]) > 2:
                        combined_code = clean_result[i] + clean_result[i + 1][:2]
                        matches.append(combined_code)
                        i += 1
                elif re.match(r"^[A-C][A-Z0-9]{1}$", clean_result[i]):
                    if i + 1 < len(clean_result) and re.match(
                        r"^[A-Z0-9]{1}$", clean_result[i + 1]
                    ):
                        combined_code = clean_result[i] + clean_result[i + 1]
                        matches.append(combined_code)
                        i += 1
                elif clean_result[i] in ["A", "B", "C", "4"]:
                    if (
                        i + 2 < len(clean_result)
                        and re.match(r"^[A-Z0-9]$", clean_result[i + 1])
                        and re.match(r"^[A-Z0-9]$", clean_result[i + 2])
                    ):
                        combined_code = (
                            clean_result[i] + clean_result[i + 1] + clean_result[i + 2]
                        )
                        matches.append(combined_code)
                        i += 2
                elif re.match(r"^[A-D][A-Z0-9]{2}$", clean_result[i]):
                    matches.append(clean_result[i])
                i += 1

            print(f"Found {len(matches)} error codes in {image_path}")
            return matches
    return []


# Excel'e sonuç yazma
def write_to_excel(excel_path, folder_name, error_counts):
    # Eğer Excel dosyası yoksa oluştur
    if not os.path.exists(excel_path):
        workbook = Workbook()
        workbook.save(excel_path)

    workbook = Workbook() if not os.path.exists(excel_path) else Workbook(excel_path)
    sheet = workbook.create_sheet(folder_name)

    # Hata kodlarını Excel'e yaz
    for i, (code, count) in enumerate(error_counts.items(), start=1):
        sheet[f"A{i}"] = code
        sheet[f"B{i}"] = count

    workbook.save(excel_path)


# Klasörleri dolaş ve her klasördeki görselleri işle
def process_folder(folder_path, excel_path):
    error_counter = Counter()
    image_files = list(Path(folder_path).rglob("*.png")) + list(
        Path(folder_path).rglob("*.jpg")
    )

    with ThreadPoolExecutor(max_workers=16) as executor:
        results = executor.map(process_image, image_files)

    # Hata kodlarını topla
    for result in results:
        error_counter.update(result)

    # Hata kodlarını Excel'e yaz
    if error_counter:
        write_to_excel(excel_path, Path(folder_path).name, error_counter)


# Ana fonksiyon: Klasörleri recursif olarak dolaş ve her klasör için işlemleri yap
def main(main_folder, excel_path):
    for root, dirs, files in os.walk(main_folder):
        if any(file.endswith(("png", "jpg")) for file in files):
            process_folder(root, excel_path)


if __name__ == "__main__":
    main(main_folder, excel_path)

TypeError: unsupported operand type(s) for +: 'generator' and 'list'